# Build The Neural Network

- The torch.nn namespace provides all the buliding blocks you need to build your own neural network
- A neural network is a module itself that consists of other modules(layers)
- This nested structure allows for building and managing complex architectures easily

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Get Device for Training

In [ ]:
# GPU 사용가능한지 확인 하고 사용하기
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Using {device} device')

Using cuda device


# Define the Class

- every nn.Module subclass implements the operations on input data in the [forward] method

In [ ]:
class NeuralNetwork(nn.Module):
  # nn.Module: for subclassing
  def __init__(self):
    # super() : 
      # allows to avoid using the base class name explicitly
      # working with multiple inheritance
      # https://www.programiz.com/python-programming/methods/built-in/super
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [ ]:
# creating an instance of [NeuralNetwork] 
# then move it to the [device]

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
X = torch.rand(1, 28, 28, device=device) # input data

# To use the model, pass it the input data : this executes the model's [forward]
# Do not call [model.forward()] directly!
logits = model(X)

pred_probab = nn.Softmax(dim=1)(logits) # getting the prediction probability
y_pred = pred_probab.argmax(1)
# argmax: finds the argument that gives the maximum value from a target function
print(f'Predicted class: {y_pred}')

Predicted class: tensor([3], device='cuda:0')


In [ ]:
pred_probab

tensor([[0.0965, 0.1068, 0.0966, 0.1116, 0.0925, 0.0974, 0.1046, 0.0982, 0.0975,
         0.0983]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [ ]:
pred_probab.argmax(1)

tensor([3], device='cuda:0')

# Model Layers

In [ ]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten

- to convert each 2D 28*28 image into a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained)

In [ ]:
flatten = nn.Flatten() # to covert each 2D (28*28) image into a contiguous array of 784 pixel values
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear


In [ ]:
layer1 = nn.Linear(in_features = 28*28, out_features = 20) 
# nn.Linear: applies a linear transformation on the input (using its stored weights and biases)

hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU


- 비선형 활성(Non-linear activations)은 모델의 input과 output 간의 복잡한 mapping들을 만드는 요소

- 비선형 활성은 선형 변형(linear transformations) 이후에 비선형성(nonlinearity)을 introduce하기 위해 적용됨 -> 신경망이 다양한 현상을 학습할 수 있도록 함

- In this model, we use nn.ReLU between our linear layers, but there's other activations to introduce non-linearity in your model 
  - ReLU: 비선형 활성(non linear activation)

- non-libearity(비선형성): 1개의 직선으로 표현되지 않는 모든 형태 == 1차 방정식으로 표현되지 않는 모든 형태

In [ ]:
print(f'Before ReLU: {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU: {hidden1}')

Before ReLU: tensor([[ 0.6590,  0.6212, -0.6310,  0.2559,  0.5597, -0.1678, -0.1582,  0.4353,
         -0.8096,  0.6206, -0.4438,  0.1359,  0.3057,  0.6956, -0.4879,  0.1492,
          0.5142, -0.1463, -0.3820, -0.3474],
        [ 0.6044,  0.7499, -0.2938, -0.1115,  0.9188, -0.0321, -0.0852,  0.3037,
         -0.6456,  0.4707, -0.4277,  0.1986, -0.0249,  0.5441, -0.4411,  0.2857,
          0.0516, -0.0525, -0.2889, -0.3182],
        [ 0.3535,  0.7041, -0.6683,  0.1864,  0.3669,  0.3178, -0.0961,  0.3621,
         -0.3924,  0.3958, -0.2389,  0.5681,  0.0618,  0.2344, -0.7747, -0.0603,
          0.0319,  0.0248, -0.3094, -0.1885]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.6590, 0.6212, 0.0000, 0.2559, 0.5597, 0.0000, 0.0000, 0.4353, 0.0000,
         0.6206, 0.0000, 0.1359, 0.3057, 0.6956, 0.0000, 0.1492, 0.5142, 0.0000,
         0.0000, 0.0000],
        [0.6044, 0.7499, 0.0000, 0.0000, 0.9188, 0.0000, 0.0000, 0.3037, 0.0000,
         0.4707, 0.0000, 0.1986, 0.0000, 0.5441, 0.00

## nn.Sequential

- nn.Sequential: 순서가 있는 모듈들의 컨테이터
- 데이터는 정해진 순서대로 각 모듈을 통과
- can use sequential containers to put together a quick network like [seq_modules]

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

## nn.Softmax


In [ ]:
softmax = nn.Softmax(dim=1)
# dim =1 : indicates the dimension along with the values must sum to 1

# logits: return of the last linear layer of the NN
# The logits are scaled to values [0,1] representing the model's predicted probabilities for each class
pred_probab = softmax(logits)

# Model Parameters

In [ ]:
print(f'Model structure: {model}\n\n')

for name, param in model.named_parameters():
  print(f'Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n')

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0264,  0.0308, -0.0301,  ..., -0.0315,  0.0253,  0.0204],
        [ 0.0282, -0.0333, -0.0340,  ...,  0.0148,  0.0184, -0.0259]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0176,  0.0353], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0259,  0.0107, -0.0020,  ...,  0.0167, -0.0398, -0.0409],
        [-0.0297,  0.0254,  0.0104,  ..., -0.0201,  0.0274,  0.0007]],
       device='cuda:0', grad_fn=<Sl